In [1]:
import pinecone 

In [2]:
API_KEY = "62ae9e89-5398-46ab-957d-1c779fc3a9fc"

In [23]:
import os
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from pathlib import Path
import json

MODEL = "mpnetv3"
# This is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.
# This gets the bi-encoding and similarity (given 1, compare to N) - this is the retriever model
MODEL_URL = "flax-sentence-embeddings/all_datasets_v3_mpnet-base"
MODEL_URL2 = "all-distilroberta-v1"
INDEX = 'search-webinar'

In [24]:
retriever_mpnet = retriever

In [25]:
retriever = SentenceTransformer(MODEL_URL)

In [10]:
# This is the actual Question answering model, given context (this must be called a reader model)
READER = "deepset/electra-base-squad2"
reader = pipeline(
    'question-answering',
    model=READER,
    tokenizer=READER
)

Downloading:   0%|          | 0.00/635 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
pinecone.init(
    api_key = API_KEY,
    environment="us-west1-gcp"
)

In [15]:
index = pinecone.Index(INDEX)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 600}},
 'total_vector_count': 600}

In [26]:
query = "How do I create custom components"
xq = retriever.encode([query]).tolist()

In [27]:
print(len(xq[0])) # we got a 768 dimension vector back

768


In [28]:
xc = index.query(
    xq, top_k=5, include_metadata=True
)

In [30]:
xc # mpnet and roberta are giving similar matches (both)

{'matches': [{'id': '51',
              'metadata': {'char_length': 79.0,
                           'is_duplicate': 0.0,
                           'tokens': ['what',
                                      'are',
                                      'some',
                                      'tips',
                                      'on',
                                      'making',
                                      'it',
                                      'through',
                                      'the',
                                      'job',
                                      'interview',
                                      'process',
                                      'at',
                                      'medicines',
                                      '?']},
              'score': 0.126125678,
              'sparseValues': {},
              'values': []},
             {'id': '460',
              'metadata': {'char_length': 47.0,
      

In [44]:
[x["id"] for x in xc["matches"]]

['51', '460', '100', '159', '391']

In [ ]:
# 